In [1]:
import math

from pycaret.classification import *
import numpy as np
import pandas as pd
import tensorflow as tf

#importing datasets
#I will be using the two smaller datasets provided for training and testing (explain why I used these in the thesis)
pathTesting = 'C:\\Users\\Jovana\\PycharmProjects\\neural_network_DNN\\include\\UNSW-NB15\\UNSW_NB15_testing-set.csv'
pathTraining= 'C:\\Users\\Jovana\\PycharmProjects\\neural_network_DNN\\include\\UNSW-NB15\\UNSW_NB15_training-set.csv'

ds_train=pd.read_csv(pathTraining)
ds_test=pd.read_csv(pathTesting)

ds=pd.concat([ds_train,ds_test], axis=0)
ds

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,254,1.415894e+04,8495.365234,0,0,24.295600,8.375000,30.177547,11.830604,255,621772692,2202533631,255,0.000000,0.000000,0.000000,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,252,8.395112e+03,503571.312500,2,17,49.915000,15.432865,61.426934,1387.778330,255,1417884146,3077387971,255,0.000000,0.000000,0.000000,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,252,1.572272e+03,60929.230470,1,6,231.875571,102.737203,17179.586860,11420.926230,255,2116150707,2963114973,255,0.111897,0.061458,0.050439,46,824,0,0,7,1,2,1,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2.740179e+03,3358.622070,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0.000000,0.000000,0.000000,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,252,8.561499e+03,3987.059814,2,1,47.750333,75.659602,2415.837634,115.807000,255,2436137549,1977154190,255,0.128381,0.071147,0.057234,53,45,0,0,43,1,2,2,1,40,0,0,0,2,39,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,82328,0.000005,udp,-,INT,2,0,104,0,200000.005100,254,0,8.320000e+07,0.000000,0,0,0.005000,0.000000,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0.000000,52,0,0,0,1,2,2,1,1,2,0,0,0,2,1,0,Normal,0
82328,82329,1.106101,tcp,-,FIN,20,8,18062,354,24.410067,254,252,1.241044e+05,2242.109863,7,1,55.880051,143.700000,4798.130981,190.980813,255,1072535109,3284291478,255,0.173208,0.100191,0.073017,903,44,0,0,1,1,2,1,1,1,0,0,0,3,2,0,Normal,0
82329,82330,0.000000,arp,-,INT,1,0,46,0,0.000000,0,0,0.000000e+00,0.000000,0,0,60000.720000,0.000000,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0.000000,46,0,0,0,1,2,1,1,1,1,0,0,0,1,1,1,Normal,0
82330,82331,0.000000,arp,-,INT,1,0,46,0,0.000000,0,0,0.000000e+00,0.000000,0,0,60000.732000,0.000000,10.954518,0.000000,0,0,0,0,0.000000,0.000000,0.000000,46,0,0,0,1,2,1,1,1,1,0,0,0,1,1,1,Normal,0


In [2]:
#there are no missing values, the feature 'service' has - where there is no service, rename it to 'undefined'
ds['service'].replace('-', 'undefined', inplace=True)
ds['label'].replace(1, 'attack', inplace=True)
ds['label'].replace(0, 'normal', inplace=True)
ds

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,undefined,FIN,6,4,258,172,74.087490,252,254,1.415894e+04,8495.365234,0,0,24.295600,8.375000,30.177547,11.830604,255,621772692,2202533631,255,0.000000,0.000000,0.000000,43,43,0,0,1,0,1,1,1,1,0,0,0,1,1,0,Normal,normal
1,2,0.649902,tcp,undefined,FIN,14,38,734,42014,78.473372,62,252,8.395112e+03,503571.312500,2,17,49.915000,15.432865,61.426934,1387.778330,255,1417884146,3077387971,255,0.000000,0.000000,0.000000,52,1106,0,0,43,1,1,1,1,2,0,0,0,1,6,0,Normal,normal
2,3,1.623129,tcp,undefined,FIN,8,16,364,13186,14.170161,62,252,1.572272e+03,60929.230470,1,6,231.875571,102.737203,17179.586860,11420.926230,255,2116150707,2963114973,255,0.111897,0.061458,0.050439,46,824,0,0,7,1,2,1,1,3,0,0,0,2,6,0,Normal,normal
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,252,2.740179e+03,3358.622070,1,3,152.876547,90.235726,259.080172,4991.784669,255,1107119177,1047442890,255,0.000000,0.000000,0.000000,52,64,0,0,1,1,2,1,1,3,1,1,0,2,1,0,Normal,normal
4,5,0.449454,tcp,undefined,FIN,10,6,534,268,33.373826,254,252,8.561499e+03,3987.059814,2,1,47.750333,75.659602,2415.837634,115.807000,255,2436137549,1977154190,255,0.128381,0.071147,0.057234,53,45,0,0,43,1,2,2,1,40,0,0,0,2,39,0,Normal,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,82328,0.000005,udp,undefined,INT,2,0,104,0,200000.005100,254,0,8.320000e+07,0.000000,0,0,0.005000,0.000000,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0.000000,52,0,0,0,1,2,2,1,1,2,0,0,0,2,1,0,Normal,normal
82328,82329,1.106101,tcp,undefined,FIN,20,8,18062,354,24.410067,254,252,1.241044e+05,2242.109863,7,1,55.880051,143.700000,4798.130981,190.980813,255,1072535109,3284291478,255,0.173208,0.100191,0.073017,903,44,0,0,1,1,2,1,1,1,0,0,0,3,2,0,Normal,normal
82329,82330,0.000000,arp,undefined,INT,1,0,46,0,0.000000,0,0,0.000000e+00,0.000000,0,0,60000.720000,0.000000,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0.000000,46,0,0,0,1,2,1,1,1,1,0,0,0,1,1,1,Normal,normal
82330,82331,0.000000,arp,undefined,INT,1,0,46,0,0.000000,0,0,0.000000e+00,0.000000,0,0,60000.732000,0.000000,10.954518,0.000000,0,0,0,0,0.000000,0.000000,0.000000,46,0,0,0,1,2,1,1,1,1,0,0,0,1,1,1,Normal,normal


In [3]:
pd.unique(ds.proto.values)

array(['tcp', 'udp', 'arp', 'ospf', 'icmp', 'igmp', 'rtp', 'ddp',
       'ipv6-frag', 'cftp', 'wsn', 'pvp', 'wb-expak', 'mtp', 'pri-enc',
       'sat-mon', 'cphb', 'sun-nd', 'iso-ip', 'xtp', 'il', 'unas',
       'mfe-nsp', '3pc', 'ipv6-route', 'idrp', 'bna', 'swipe',
       'kryptolan', 'cpnx', 'rsvp', 'wb-mon', 'vmtp', 'ib', 'dgp',
       'eigrp', 'ax.25', 'gmtp', 'pnni', 'sep', 'pgm', 'idpr-cmtp',
       'zero', 'rvd', 'mobile', 'narp', 'fc', 'pipe', 'ipcomp', 'ipv6-no',
       'sat-expak', 'ipv6-opts', 'snp', 'ipcv', 'br-sat-mon', 'ttp',
       'tcf', 'nsfnet-igp', 'sprite-rpc', 'aes-sp3-d', 'sccopmce', 'sctp',
       'qnx', 'scps', 'etherip', 'aris', 'pim', 'compaq-peer', 'vrrp',
       'iatp', 'stp', 'l2tp', 'srp', 'sm', 'isis', 'smp', 'fire', 'ptp',
       'crtp', 'sps', 'merit-inp', 'idpr', 'skip', 'any', 'larp', 'ipip',
       'micp', 'encap', 'ifmp', 'tp++', 'a/n', 'ipv6', 'i-nlsp',
       'ipx-n-ip', 'sdrp', 'tlsp', 'gre', 'mhrp', 'ddx', 'ippc', 'visa',
       'secure-vmtp', 

In [4]:
#drop the id column
ds = ds.drop('id', axis=1)
#first binary classification, for that we will drop the attack cat
ds_binary = ds.drop('attack_cat', axis=1)

In [5]:
#first normal without smote or anything
dsp_tr_binary=setup(data = ds_binary,
             target = 'label',train_size=0.8,
             numeric_imputation = 'mean',
             categorical_features = ['proto','service','state'],
             numeric_features=['dur','spkts','dpkts','sbytes','dbytes','rate','sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean','trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm',
                               'ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports'],
             normalize=True,normalize_method='minmax',
             feature_selection=True,feature_selection_threshold=0.2,
             silent = True, fold_shuffle=True)

,Description,Value
0,session_id,6499
1,Target,label
2,Target Type,Binary
3,Label Encoded,"attack: 0, normal: 1"
4,Original Data,"(257673, 43)"
5,Missing Values,False
6,Numeric Features,39
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


In [6]:
X_train_bin=get_config('X_train')
X_test_bin=get_config('X_test')
Y_train_bin=get_config('y_train')
Y_test_bin=get_config('y_test')

In [16]:
Y_train_bin

17340     1
114826    1
96988     0
146949    0
125551    0
         ..
37873     1
17587     0
136826    0
66308     0
859       0
Name: label, Length: 206138, dtype: int32

In [17]:
#reverse 1 and 0
Y_train_bin=Y_train_bin.replace({0:1,1:0})

In [18]:
Y_train_bin

17340     0
114826    0
96988     1
146949    1
125551    1
         ..
37873     0
17587     1
136826    1
66308     1
859       1
Name: label, Length: 206138, dtype: int64

In [19]:
Y_test_bin=Y_test_bin.replace({0:1,1:0})

In [21]:
Y_test_bin

136799    1
57666     1
78136     0
51229     1
6822      1
         ..
32538     0
29521     0
46088     0
133452    1
86469     1
Name: label, Length: 51535, dtype: int64

In [20]:
#now we make and test the neural networks
from tensorflow.python.keras.layers import Activation
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.advanced_activations import PReLU
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
act2 = PReLU(alpha_initializer='zeros', weights=None)
model2 = Sequential()
# define the NN architecture and activation function

model2.add(Dense(496, input_dim=52, activation=Activation(act2)))
model2.add(Dropout(0.1))
model2.add(Dense(248, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(124, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(62, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(1))
model2.add(Activation('sigmoid'))

sgd =SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model2.compile(loss='binary_crossentropy', optimizer=sgd, metrics= ['accuracy'])
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.0001,patience=5) ## early stopping
model2.fit(X_train_bin, Y_train_bin, validation_data=(X_test_bin, Y_test_bin), epochs=100, batch_size=64, callbacks=[es])

Epoch 1/100
3221/3221 [==============================] - 10s 3ms/step - loss: 0.1977 - accuracy: 0.8948 - val_loss: 0.1718 - val_accuracy: 0.9079
Epoch 2/100
3221/3221 [==============================] - 11s 3ms/step - loss: 0.1720 - accuracy: 0.9055 - val_loss: 0.1636 - val_accuracy: 0.9110
Epoch 3/100
3221/3221 [==============================] - 11s 3ms/step - loss: 0.1683 - accuracy: 0.9081 - val_loss: 0.1669 - val_accuracy: 0.9071
Epoch 4/100
3221/3221 [==============================] - 10s 3ms/step - loss: 0.1663 - accuracy: 0.9091 - val_loss: 0.1571 - val_accuracy: 0.9160
Epoch 5/100
3221/3221 [==============================] - 11s 3ms/step - loss: 0.1648 - accuracy: 0.9104 - val_loss: 0.1583 - val_accuracy: 0.9144
Epoch 6/100
3221/3221 [==============================] - 11s 3ms/step - loss: 0.1634 - accuracy: 0.9116 - val_loss: 0.1570 - val_accuracy: 0.9134
Epoch 7/100
3221/3221 [==============================] - 11s 3ms/step - loss: 0.1625 - accuracy: 0.9121 - val_loss: 0.1529 -

In [22]:
from sklearn.metrics import accuracy_score
# or find the prediction accuracy using predict_classes (same as test accuracy)
y_pred2 = model2.predict_classes(X_test_bin)
pred_accuracy2 = accuracy_score(Y_test_bin, y_pred2)
pred_accuracy2

0.9254099155913457

In [23]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
# Precision
precision = precision_score(Y_test_bin, y_pred2)
print('Precision : %f'% precision)

Precision : 0.944497


In [24]:
# Recall
recall = recall_score(Y_test_bin, y_pred2)
print('Recall: %f' % recall)

Recall: 0.938115


In [25]:
# f1-score
f1 = f1_score(Y_test_bin, y_pred2)
print('F1 score: %f' % f1)

F1 score: 0.941295


In [13]:
roc = roc_auc_score(Y_test_bin, y_pred2)
print('ROC_AUC Score: %f' % roc)

ROC_AUC Score: 0.923796


In [26]:
from sklearn.metrics import confusion_matrix

# confusion matrix
confusionmatrix = confusion_matrix(Y_test_bin, y_pred2)
print(confusionmatrix)

[[16873  1811]
 [ 2033 30818]]


In [27]:
tn, fp, fn, tp = confusionmatrix.ravel()
(tn, fp, fn, tp)

(16873, 1811, 2033, 30818)

In [28]:
Y_train_bin

17340     0
114826    0
96988     1
146949    1
125551    1
         ..
37873     0
17587     1
136826    1
66308     1
859       1
Name: label, Length: 206138, dtype: int64

In [29]:
Y_test_bin

136799    1
57666     1
78136     0
51229     1
6822      1
         ..
32538     0
29521     0
46088     0
133452    1
86469     1
Name: label, Length: 51535, dtype: int64

In [47]:
# I should try with the full dataset
### FULL DATASET ###
path1="D:\\Master Thesis Datasets\\UNSW-NB15 Source-Files\\UNSW-NB15 - CSV Files\\UNSW-NB15_1.csv"
path2="D:\\Master Thesis Datasets\\UNSW-NB15 Source-Files\\UNSW-NB15 - CSV Files\\UNSW-NB15_2.csv"
path3="D:\\Master Thesis Datasets\\UNSW-NB15 Source-Files\\UNSW-NB15 - CSV Files\\UNSW-NB15_3.csv"
path4="D:\\Master Thesis Datasets\\UNSW-NB15 Source-Files\\UNSW-NB15 - CSV Files\\UNSW-NB15_4.csv"
columns = ['srcip','sport','dstip','dsport','proto','state','dur','sbytes','dbytes','sttl','dttl','sloss','dloss',
                   'service','Sload','Dload','Spkts','Dpkts','swin','dwin','stcpb','dtcpb','smeansz','dmeansz','trans_depth',
                   'res_bdy_len','Sjit','Djit','Stime','Ltime','Sintpkt','Dintpkt','tcprtt','synack','ackdat','is_sm_ips_ports',
                   'ct_state_ttl','ct_flw_http_mthd','is_ftp_login','ct_ftp_cmd','ct_srv_src','ct_srv_dst','ct_dst_ltm','ct_src_ ltm',
                   'ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','attack_cat','label']

ds1=pd.read_csv(path1)
ds1.columns=columns
ds2=pd.read_csv(path2)
ds2.columns=columns
ds3=pd.read_csv(path3)
ds3.columns=columns
ds4=pd.read_csv(path4)
ds4.columns=columns
ds_full=pd.concat([ds1,ds2,ds3,ds4], axis=0)
ds_full.columns

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
       'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime',
       'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
       'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat',
       'label'],
      dtype='object')

In [48]:
ds_full.to_csv("D:\\Master Thesis Datasets\\UNSW-NB15 Source-Files\\UNSW-NB15 - CSV Files\\FULL.csv")

In [2]:
import math

from pycaret.classification import *
import numpy as np
import pandas as pd
import tensorflow as tf
ds_full=pd.read_csv("D:\\Master Thesis Datasets\\UNSW-NB15 Source-Files\\UNSW-NB15 - CSV Files\\FULL.csv")
ds_full

,Unnamed: 0,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,label
0,0,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,29,0,0,-,87676.085940,50480.171880,4,4,0,0,0,0,132,76,0,0,9.891010,10.682733,1421927414,1421927414,7.005000,7.564333,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,NaN,0
1,1,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.531300,636282.375000,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.017000,0.013000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,12,8,1,2,2,1,1,NaN,0
2,2,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.562500,542597.187500,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,1421927414,1421927414,0.043000,0.014000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,6,9,1,1,1,1,1,NaN,0
3,3,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.250000,609067.562500,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.005000,0.003000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,7,9,1,1,1,1,1,NaN,0
4,4,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,29,0,0,-,43503.234380,23896.142580,4,4,0,0,0,0,142,78,0,0,29.682221,34.370340,1421927414,1421927414,21.003000,24.315000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540038,440038,59.166.0.5,33094,149.171.126.7,43433,tcp,FIN,0.087306,320,1828,31,29,1,2,ftp-data,24465.671880,146610.765600,6,8,255,255,4035116044,222196311,53,229,0,0,1221.136547,30.096029,1424262068,1424262069,17.394400,12.405143,0.000608,0.000468,0.000140,0,0,NaN,NaN,,1,2,3,3,1,1,3,NaN,0
2540039,440039,59.166.0.7,20848,149.171.126.4,21,tcp,CON,0.365058,456,346,31,29,2,2,ftp,8743.816406,6333.240234,8,6,255,255,1529727621,185369967,57,58,0,0,3961.093354,129.038367,1424262068,1424262069,51.976999,72.915602,0.000609,0.000477,0.000132,0,0,NaN,2.0,2,2,2,2,2,2,2,2,NaN,0
2540040,440040,59.166.0.3,21511,149.171.126.9,21,tcp,CON,6.335154,1802,2088,31,29,7,9,ftp,2204.839844,2549.582764,32,30,255,255,497208793,889904111,56,70,0,0,13233.728560,379.052750,1424262062,1424262069,204.359814,214.372375,0.000617,0.000485,0.000132,0,0,NaN,2.0,2,2,2,4,2,2,2,2,NaN,0
2540041,440041,59.166.0.9,35433,149.171.126.0,80,tcp,CON,2.200934,3498,166054,31,29,2,57,http,12496.513670,598375.062500,58,116,255,255,1289728069,3436097151,60,1432,1,79745,2868.893137,1951.035944,1424262067,1424262069,38.609700,19.133826,0.000680,0.000541,0.000139,0,0,2.0,NaN,,1,1,2,4,2,2,2,NaN,0


In [3]:
ds_full.drop(columns="Unnamed: 0",inplace=True)
ds_full

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,label
0,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,29,0,0,-,87676.085940,50480.171880,4,4,0,0,0,0,132,76,0,0,9.891010,10.682733,1421927414,1421927414,7.005000,7.564333,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,NaN,0
1,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.531300,636282.375000,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.017000,0.013000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,12,8,1,2,2,1,1,NaN,0
2,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.562500,542597.187500,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,1421927414,1421927414,0.043000,0.014000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,6,9,1,1,1,1,1,NaN,0
3,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.250000,609067.562500,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.005000,0.003000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,7,9,1,1,1,1,1,NaN,0
4,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,29,0,0,-,43503.234380,23896.142580,4,4,0,0,0,0,142,78,0,0,29.682221,34.370340,1421927414,1421927414,21.003000,24.315000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540038,59.166.0.5,33094,149.171.126.7,43433,tcp,FIN,0.087306,320,1828,31,29,1,2,ftp-data,24465.671880,146610.765600,6,8,255,255,4035116044,222196311,53,229,0,0,1221.136547,30.096029,1424262068,1424262069,17.394400,12.405143,0.000608,0.000468,0.000140,0,0,NaN,NaN,,1,2,3,3,1,1,3,NaN,0
2540039,59.166.0.7,20848,149.171.126.4,21,tcp,CON,0.365058,456,346,31,29,2,2,ftp,8743.816406,6333.240234,8,6,255,255,1529727621,185369967,57,58,0,0,3961.093354,129.038367,1424262068,1424262069,51.976999,72.915602,0.000609,0.000477,0.000132,0,0,NaN,2.0,2,2,2,2,2,2,2,2,NaN,0
2540040,59.166.0.3,21511,149.171.126.9,21,tcp,CON,6.335154,1802,2088,31,29,7,9,ftp,2204.839844,2549.582764,32,30,255,255,497208793,889904111,56,70,0,0,13233.728560,379.052750,1424262062,1424262069,204.359814,214.372375,0.000617,0.000485,0.000132,0,0,NaN,2.0,2,2,2,4,2,2,2,2,NaN,0
2540041,59.166.0.9,35433,149.171.126.0,80,tcp,CON,2.200934,3498,166054,31,29,2,57,http,12496.513670,598375.062500,58,116,255,255,1289728069,3436097151,60,1432,1,79745,2868.893137,1951.035944,1424262067,1424262069,38.609700,19.133826,0.000680,0.000541,0.000139,0,0,2.0,NaN,,1,1,2,4,2,2,2,NaN,0


In [4]:
# see how many duplicates
ds_full.duplicated().sum()

480625

In [5]:
#remove these duplicates
ds_full_new=ds_full.drop_duplicates()
ds_full_new

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,label
0,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,29,0,0,-,87676.085940,5.048017e+04,4,4,0,0,0,0,132,76,0,0,9.891010,10.682733,1421927414,1421927414,7.005000,7.564333,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,NaN,0
1,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.531300,6.362824e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.017000,0.013000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,12,8,1,2,2,1,1,NaN,0
2,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.562500,5.425972e+05,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,1421927414,1421927414,0.043000,0.014000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,6,9,1,1,1,1,1,NaN,0
3,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.250000,6.090676e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.005000,0.003000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,7,9,1,1,1,1,1,NaN,0
4,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,29,0,0,-,43503.234380,2.389614e+04,4,4,0,0,0,0,142,78,0,0,29.682221,34.370340,1421927414,1421927414,21.003000,24.315000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540035,59.166.0.1,38606,149.171.126.9,80,tcp,CON,0.564998,14106,772406,31,29,2,262,http,198981.250000,1.091598e+07,262,526,255,255,162243658,163257853,54,1468,1,372266,116.321921,69.874077,1424262068,1424262069,2.123130,1.075171,0.000669,0.000530,0.000139,0,0,2.0,NaN,,1,1,4,2,2,2,2,NaN,0
2540037,59.166.0.1,38606,149.171.126.9,80,tcp,CON,0.564998,14106,772406,31,29,2,262,-,198981.250000,1.091598e+07,262,526,255,255,162243658,163257853,54,1468,1,372266,116.321921,69.874077,1424262068,1424262069,2.123130,1.075171,0.000669,0.000530,0.000139,0,0,2.0,NaN,,2,1,4,2,2,2,2,NaN,0
2540038,59.166.0.5,33094,149.171.126.7,43433,tcp,FIN,0.087306,320,1828,31,29,1,2,ftp-data,24465.671880,1.466108e+05,6,8,255,255,4035116044,222196311,53,229,0,0,1221.136547,30.096029,1424262068,1424262069,17.394400,12.405143,0.000608,0.000468,0.000140,0,0,NaN,NaN,,1,2,3,3,1,1,3,NaN,0
2540041,59.166.0.9,35433,149.171.126.0,80,tcp,CON,2.200934,3498,166054,31,29,2,57,http,12496.513670,5.983751e+05,58,116,255,255,1289728069,3436097151,60,1432,1,79745,2868.893137,1951.035944,1424262067,1424262069,38.609700,19.133826,0.000680,0.000541,0.000139,0,0,2.0,NaN,,1,1,2,4,2,2,2,NaN,0


In [6]:
# replace "-" with "undefined"
ds_full_new['service'].replace('-','undefined',inplace=True)

In [7]:
ds_full_new

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,label
0,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,29,0,0,undefined,87676.085940,5.048017e+04,4,4,0,0,0,0,132,76,0,0,9.891010,10.682733,1421927414,1421927414,7.005000,7.564333,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,NaN,0
1,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.531300,6.362824e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.017000,0.013000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,12,8,1,2,2,1,1,NaN,0
2,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.562500,5.425972e+05,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,1421927414,1421927414,0.043000,0.014000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,6,9,1,1,1,1,1,NaN,0
3,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.250000,6.090676e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.005000,0.003000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,7,9,1,1,1,1,1,NaN,0
4,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,29,0,0,undefined,43503.234380,2.389614e+04,4,4,0,0,0,0,142,78,0,0,29.682221,34.370340,1421927414,1421927414,21.003000,24.315000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540035,59.166.0.1,38606,149.171.126.9,80,tcp,CON,0.564998,14106,772406,31,29,2,262,http,198981.250000,1.091598e+07,262,526,255,255,162243658,163257853,54,1468,1,372266,116.321921,69.874077,1424262068,1424262069,2.123130,1.075171,0.000669,0.000530,0.000139,0,0,2.0,NaN,,1,1,4,2,2,2,2,NaN,0
2540037,59.166.0.1,38606,149.171.126.9,80,tcp,CON,0.564998,14106,772406,31,29,2,262,undefined,198981.250000,1.091598e+07,262,526,255,255,162243658,163257853,54,1468,1,372266,116.321921,69.874077,1424262068,1424262069,2.123130,1.075171,0.000669,0.000530,0.000139,0,0,2.0,NaN,,2,1,4,2,2,2,2,NaN,0
2540038,59.166.0.5,33094,149.171.126.7,43433,tcp,FIN,0.087306,320,1828,31,29,1,2,ftp-data,24465.671880,1.466108e+05,6,8,255,255,4035116044,222196311,53,229,0,0,1221.136547,30.096029,1424262068,1424262069,17.394400,12.405143,0.000608,0.000468,0.000140,0,0,NaN,NaN,,1,2,3,3,1,1,3,NaN,0
2540041,59.166.0.9,35433,149.171.126.0,80,tcp,CON,2.200934,3498,166054,31,29,2,57,http,12496.513670,5.983751e+05,58,116,255,255,1289728069,3436097151,60,1432,1,79745,2868.893137,1951.035944,1424262067,1424262069,38.609700,19.133826,0.000680,0.000541,0.000139,0,0,2.0,NaN,,1,1,2,4,2,2,2,NaN,0


In [8]:
#change NaN to Normal
import math
ds_full_new['attack_cat'].replace(math.nan,'Normal',inplace=True)

In [9]:
ds_full_new

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,label
0,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,29,0,0,undefined,87676.085940,5.048017e+04,4,4,0,0,0,0,132,76,0,0,9.891010,10.682733,1421927414,1421927414,7.005000,7.564333,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,Normal,0
1,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.531300,6.362824e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.017000,0.013000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,12,8,1,2,2,1,1,Normal,0
2,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.562500,5.425972e+05,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,1421927414,1421927414,0.043000,0.014000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,6,9,1,1,1,1,1,Normal,0
3,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.250000,6.090676e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.005000,0.003000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,7,9,1,1,1,1,1,Normal,0
4,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,29,0,0,undefined,43503.234380,2.389614e+04,4,4,0,0,0,0,142,78,0,0,29.682221,34.370340,1421927414,1421927414,21.003000,24.315000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540035,59.166.0.1,38606,149.171.126.9,80,tcp,CON,0.564998,14106,772406,31,29,2,262,http,198981.250000,1.091598e+07,262,526,255,255,162243658,163257853,54,1468,1,372266,116.321921,69.874077,1424262068,1424262069,2.123130,1.075171,0.000669,0.000530,0.000139,0,0,2.0,NaN,,1,1,4,2,2,2,2,Normal,0
2540037,59.166.0.1,38606,149.171.126.9,80,tcp,CON,0.564998,14106,772406,31,29,2,262,undefined,198981.250000,1.091598e+07,262,526,255,255,162243658,163257853,54,1468,1,372266,116.321921,69.874077,1424262068,1424262069,2.123130,1.075171,0.000669,0.000530,0.000139,0,0,2.0,NaN,,2,1,4,2,2,2,2,Normal,0
2540038,59.166.0.5,33094,149.171.126.7,43433,tcp,FIN,0.087306,320,1828,31,29,1,2,ftp-data,24465.671880,1.466108e+05,6,8,255,255,4035116044,222196311,53,229,0,0,1221.136547,30.096029,1424262068,1424262069,17.394400,12.405143,0.000608,0.000468,0.000140,0,0,NaN,NaN,,1,2,3,3,1,1,3,Normal,0
2540041,59.166.0.9,35433,149.171.126.0,80,tcp,CON,2.200934,3498,166054,31,29,2,57,http,12496.513670,5.983751e+05,58,116,255,255,1289728069,3436097151,60,1432,1,79745,2868.893137,1951.035944,1424262067,1424262069,38.609700,19.133826,0.000680,0.000541,0.000139,0,0,2.0,NaN,,1,1,2,4,2,2,2,Normal,0


In [10]:
#Ignore source IP, destination IP and attack category (for binary) features when training and testing the model.
# drop source IP and destination IP
ds_full_new.drop(columns=['srcip','dstip'],inplace=True)
ds_full_new

,sport,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,label
0,33661,1024,udp,CON,0.036133,528,304,31,29,0,0,undefined,87676.085940,5.048017e+04,4,4,0,0,0,0,132,76,0,0,9.891010,10.682733,1421927414,1421927414,7.005000,7.564333,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,Normal,0
1,1464,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.531300,6.362824e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.017000,0.013000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,12,8,1,2,2,1,1,Normal,0
2,3593,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.562500,5.425972e+05,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,1421927414,1421927414,0.043000,0.014000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,6,9,1,1,1,1,1,Normal,0
3,49664,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.250000,6.090676e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.005000,0.003000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,7,9,1,1,1,1,1,Normal,0
4,32119,111,udp,CON,0.078339,568,312,31,29,0,0,undefined,43503.234380,2.389614e+04,4,4,0,0,0,0,142,78,0,0,29.682221,34.370340,1421927414,1421927414,21.003000,24.315000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540035,38606,80,tcp,CON,0.564998,14106,772406,31,29,2,262,http,198981.250000,1.091598e+07,262,526,255,255,162243658,163257853,54,1468,1,372266,116.321921,69.874077,1424262068,1424262069,2.123130,1.075171,0.000669,0.000530,0.000139,0,0,2.0,NaN,,1,1,4,2,2,2,2,Normal,0
2540037,38606,80,tcp,CON,0.564998,14106,772406,31,29,2,262,undefined,198981.250000,1.091598e+07,262,526,255,255,162243658,163257853,54,1468,1,372266,116.321921,69.874077,1424262068,1424262069,2.123130,1.075171,0.000669,0.000530,0.000139,0,0,2.0,NaN,,2,1,4,2,2,2,2,Normal,0
2540038,33094,43433,tcp,FIN,0.087306,320,1828,31,29,1,2,ftp-data,24465.671880,1.466108e+05,6,8,255,255,4035116044,222196311,53,229,0,0,1221.136547,30.096029,1424262068,1424262069,17.394400,12.405143,0.000608,0.000468,0.000140,0,0,NaN,NaN,,1,2,3,3,1,1,3,Normal,0
2540041,35433,80,tcp,CON,2.200934,3498,166054,31,29,2,57,http,12496.513670,5.983751e+05,58,116,255,255,1289728069,3436097151,60,1432,1,79745,2868.893137,1951.035944,1424262067,1424262069,38.609700,19.133826,0.000680,0.000541,0.000139,0,0,2.0,NaN,,1,1,2,4,2,2,2,Normal,0


In [11]:
#binary without the attack_cat
ds_full_binary=ds_full_new.drop(columns='attack_cat')
ds_full_binary

,sport,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,label
0,33661,1024,udp,CON,0.036133,528,304,31,29,0,0,undefined,87676.085940,5.048017e+04,4,4,0,0,0,0,132,76,0,0,9.891010,10.682733,1421927414,1421927414,7.005000,7.564333,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,0
1,1464,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.531300,6.362824e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.017000,0.013000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,12,8,1,2,2,1,1,0
2,3593,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.562500,5.425972e+05,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,1421927414,1421927414,0.043000,0.014000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,6,9,1,1,1,1,1,0
3,49664,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.250000,6.090676e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.005000,0.003000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,7,9,1,1,1,1,1,0
4,32119,111,udp,CON,0.078339,568,312,31,29,0,0,undefined,43503.234380,2.389614e+04,4,4,0,0,0,0,142,78,0,0,29.682221,34.370340,1421927414,1421927414,21.003000,24.315000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540035,38606,80,tcp,CON,0.564998,14106,772406,31,29,2,262,http,198981.250000,1.091598e+07,262,526,255,255,162243658,163257853,54,1468,1,372266,116.321921,69.874077,1424262068,1424262069,2.123130,1.075171,0.000669,0.000530,0.000139,0,0,2.0,NaN,,1,1,4,2,2,2,2,0
2540037,38606,80,tcp,CON,0.564998,14106,772406,31,29,2,262,undefined,198981.250000,1.091598e+07,262,526,255,255,162243658,163257853,54,1468,1,372266,116.321921,69.874077,1424262068,1424262069,2.123130,1.075171,0.000669,0.000530,0.000139,0,0,2.0,NaN,,2,1,4,2,2,2,2,0
2540038,33094,43433,tcp,FIN,0.087306,320,1828,31,29,1,2,ftp-data,24465.671880,1.466108e+05,6,8,255,255,4035116044,222196311,53,229,0,0,1221.136547,30.096029,1424262068,1424262069,17.394400,12.405143,0.000608,0.000468,0.000140,0,0,NaN,NaN,,1,2,3,3,1,1,3,0
2540041,35433,80,tcp,CON,2.200934,3498,166054,31,29,2,57,http,12496.513670,5.983751e+05,58,116,255,255,1289728069,3436097151,60,1432,1,79745,2868.893137,1951.035944,1424262067,1424262069,38.609700,19.133826,0.000680,0.000541,0.000139,0,0,2.0,NaN,,1,1,2,4,2,2,2,0


In [12]:
#0 is normal, 1 is attack
#make the multiclass now
ds_full_multiclass=ds_full_new.drop(columns='label')

In [13]:
ds_full_multiclass

,sport,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat
0,33661,1024,udp,CON,0.036133,528,304,31,29,0,0,undefined,87676.085940,5.048017e+04,4,4,0,0,0,0,132,76,0,0,9.891010,10.682733,1421927414,1421927414,7.005000,7.564333,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,Normal
1,1464,53,udp,CON,0.001119,146,178,31,29,0,0,dns,521894.531300,6.362824e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.017000,0.013000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,12,8,1,2,2,1,1,Normal
2,3593,53,udp,CON,0.001209,132,164,31,29,0,0,dns,436724.562500,5.425972e+05,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,1421927414,1421927414,0.043000,0.014000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,6,9,1,1,1,1,1,Normal
3,49664,53,udp,CON,0.001169,146,178,31,29,0,0,dns,499572.250000,6.090676e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.005000,0.003000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,7,9,1,1,1,1,1,Normal
4,32119,111,udp,CON,0.078339,568,312,31,29,0,0,undefined,43503.234380,2.389614e+04,4,4,0,0,0,0,142,78,0,0,29.682221,34.370340,1421927414,1421927414,21.003000,24.315000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,2,4,2,3,1,1,2,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540035,38606,80,tcp,CON,0.564998,14106,772406,31,29,2,262,http,198981.250000,1.091598e+07,262,526,255,255,162243658,163257853,54,1468,1,372266,116.321921,69.874077,1424262068,1424262069,2.123130,1.075171,0.000669,0.000530,0.000139,0,0,2.0,NaN,,1,1,4,2,2,2,2,Normal
2540037,38606,80,tcp,CON,0.564998,14106,772406,31,29,2,262,undefined,198981.250000,1.091598e+07,262,526,255,255,162243658,163257853,54,1468,1,372266,116.321921,69.874077,1424262068,1424262069,2.123130,1.075171,0.000669,0.000530,0.000139,0,0,2.0,NaN,,2,1,4,2,2,2,2,Normal
2540038,33094,43433,tcp,FIN,0.087306,320,1828,31,29,1,2,ftp-data,24465.671880,1.466108e+05,6,8,255,255,4035116044,222196311,53,229,0,0,1221.136547,30.096029,1424262068,1424262069,17.394400,12.405143,0.000608,0.000468,0.000140,0,0,NaN,NaN,,1,2,3,3,1,1,3,Normal
2540041,35433,80,tcp,CON,2.200934,3498,166054,31,29,2,57,http,12496.513670,5.983751e+05,58,116,255,255,1289728069,3436097151,60,1432,1,79745,2868.893137,1951.035944,1424262067,1424262069,38.609700,19.133826,0.000680,0.000541,0.000139,0,0,2.0,NaN,,1,1,2,4,2,2,2,Normal


In [14]:
pd.value_counts(ds_full_multiclass.attack_cat.values)

Normal              1959772
Exploits              27600
Generic               25378
 Fuzzers              17804
 Reconnaissance       11617
DoS                    5665
 Fuzzers               3991
Analysis               2185
Reconnaissance         1740
Backdoor               1684
 Shellcode             1288
Backdoors               300
Shellcode               223
Worms                   171
dtype: int64

In [15]:
#there are some typos, have to fix
ds_full_multiclass['attack_cat']=ds_full_multiclass['attack_cat'].str.strip()

In [16]:
pd.value_counts(ds_full_multiclass.attack_cat.values)

Normal            1959772
Exploits            27600
Generic             25378
Fuzzers             21795
Reconnaissance      13357
DoS                  5665
Analysis             2185
Backdoor             1684
Shellcode            1511
Backdoors             300
Worms                 171
dtype: int64

In [17]:
ds_full_multiclass['attack_cat'].replace('Backdoors','Backdoor',inplace=True)
pd.value_counts(ds_full_multiclass.attack_cat.values)

Normal            1959772
Exploits            27600
Generic             25378
Fuzzers             21795
Reconnaissance      13357
DoS                  5665
Analysis             2185
Backdoor             1984
Shellcode            1511
Worms                 171
dtype: int64

In [18]:
pd.unique(ds_full_multiclass.attack_cat)

array(['Normal', 'Exploits', 'Reconnaissance', 'DoS', 'Generic',
       'Shellcode', 'Fuzzers', 'Worms', 'Backdoor', 'Analysis'],
      dtype=object)

In [65]:
#now we should remove whitespaces everywhere
ds_full_binary.isnull().sum()

         sport dsport proto state    service ct_ftp_cmd
0        33661   1024   udp   CON  undefined          0
1         1464     53   udp   CON        dns          0
2         3593     53   udp   CON        dns          0
3        49664     53   udp   CON        dns          0
4        32119    111   udp   CON  undefined          0
...        ...    ...   ...   ...        ...        ...
2540035  38606     80   tcp   CON       http           
2540037  38606     80   tcp   CON  undefined           
2540038  33094  43433   tcp   FIN   ftp-data           
2540041  35433     80   tcp   CON       http           
2540042  17293    110   tcp   CON       pop3           

[2059418 rows x 6 columns]


In [72]:
pd.unique(ds_full_binary.ct_ftp_cmd)

array(['0', 1, 6, 2, 4, 8, 5, 3, '1', '2', '4'], dtype=object)

In [71]:
ds_full_binary['ct_ftp_cmd'].replace(1, '1',inplace=True)

In [73]:
dsp_tr_binary_full=setup(data = ds_full_binary,
             target = 'label',train_size=0.8,
             numeric_imputation = 'mean',
             categorical_features = ['proto','service','state'],
             numeric_features=['dur','Spkts','Dpkts','sbytes','dbytes','sttl','dttl','Sload','Dload','sloss','dloss','Sintpkt','Dintpkt','Sjit','Djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smeansz','dmeansz','trans_depth','res_bdy_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm',
                               'ct_dst_src_ltm','is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ ltm','ct_srv_dst','is_sm_ips_ports'],
                ignore_features=['dsport','sport','Stime','Ltime'],
             normalize=True,normalize_method='minmax',
             feature_selection=True,feature_selection_threshold=0.2,
             silent = True, fold_shuffle=True)

,Description,Value
0,session_id,264
1,Target,label
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(2059418, 46)"
5,Missing Values,False
6,Numeric Features,38
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


In [74]:
X_train_bin_full=get_config('X_train')
X_test_bin_full = get_config('X_test')
Y_train_bin_full = get_config('y_train')
Y_test_bin_full = get_config('y_test')

In [75]:
X_train_bin_full.columns

Index(['proto_udp', 'state_CON', 'service_dns', 'dloss', 'ct_srv_src',
       'ct_dst_sport_ltm', 'proto_unas', 'ackdat', 'Dpkts', 'ct_flw_http_mthd',
       'is_sm_ips_ports', 'stcpb', 'service_undefined', 'state_REQ',
       'proto_sctp', 'proto_ospf', 'trans_depth', 'Dload', 'sloss',
       'state_INT', 'dur', 'res_bdy_len', 'sbytes', 'service_ssl', 'dbytes',
       'ct_dst_src_ltm', 'sttl', 'ct_state_ttl', 'dmeansz', 'ct_srv_dst',
       'dttl', 'service_dhcp', 'service_ssh', 'tcprtt', 'state_RST',
       'proto_any', 'Sjit', 'Spkts', 'proto_igmp', 'ct_dst_ltm', 'Djit',
       'Sload', 'synack', 'ct_src_ ltm', 'Sintpkt', 'dtcpb', 'state_FIN',
       'ct_src_dport_ltm', 'service_ftp-data', 'Dintpkt', 'service_pop3',
       'proto_tcp', 'service_http', 'smeansz'],
      dtype='object')

In [79]:
from tensorflow.python.keras.layers import Activation
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.advanced_activations import PReLU
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
act11 = PReLU(alpha_initializer='zeros', weights=None)
model11 = Sequential()
# define the NN architecture and activation function

model11.add(Dense(496, input_dim=54, activation=Activation(act11)))
model11.add(Dropout(0.1))
model11.add(Dense(248, activation='relu'))
model11.add(Dropout(0.1))
model11.add(Dense(124, activation='relu'))
model11.add(Dropout(0.1))
model11.add(Dense(62, activation='relu'))
model11.add(Dropout(0.1))
model11.add(Dense(1))
model11.add(Activation('sigmoid'))

sgd =SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model11.compile(loss='binary_crossentropy', optimizer=sgd, metrics= ['accuracy'])
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.0001,patience=5) ## early stopping
model11.fit(X_train_bin_full, Y_train_bin_full, validation_data=(X_test_bin_full, Y_test_bin_full), epochs=100, batch_size=64, callbacks=[es])

Epoch 1/100
25743/25743 [==============================] - 85s 3ms/step - loss: 0.0267 - accuracy: 0.9858 - val_loss: 0.0222 - val_accuracy: 0.9881
Epoch 2/100
25743/25743 [==============================] - 83s 3ms/step - loss: 0.0229 - accuracy: 0.9874 - val_loss: 0.0215 - val_accuracy: 0.9884
Epoch 3/100
25743/25743 [==============================] - 85s 3ms/step - loss: 0.0223 - accuracy: 0.9878 - val_loss: 0.0221 - val_accuracy: 0.9881
Epoch 4/100
25743/25743 [==============================] - 82s 3ms/step - loss: 0.0219 - accuracy: 0.9881 - val_loss: 0.0213 - val_accuracy: 0.9886
Epoch 5/100
25743/25743 [==============================] - 83s 3ms/step - loss: 0.0217 - accuracy: 0.9882 - val_loss: 0.0212 - val_accuracy: 0.9885
Epoch 6/100
25743/25743 [==============================] - 83s 3ms/step - loss: 0.0216 - accuracy: 0.9883 - val_loss: 0.0205 - val_accuracy: 0.9888
Epoch 7/100
25743/25743 [==============================] - 81s 3ms/step - loss: 0.0215 - accuracy: 0.9883 - val_

In [81]:
#training accuracy
train_acc11 = model11.evaluate(X_train_bin_full, Y_train_bin_full, verbose=0)
print("%s: %.2f%%" % (model11.metrics_names[1], train_acc11[1]*100))

accuracy: 98.88%


In [83]:
#calculate the test accuracy using evaluate function
test_acc11 = model11.evaluate(X_test_bin_full, Y_test_bin_full, verbose=0)
print("%s: %.2f%%" % (model11.metrics_names[1], test_acc11[1] * 100))

accuracy: 98.91%


In [84]:
from sklearn.metrics import accuracy_score
# or find the prediction accuracy using predict_classes (same as test accuracy)
y_pred11 = model11.predict_classes(X_test_bin_full)
pred_accuracy11 = accuracy_score(Y_test_bin_full, y_pred11)
pred_accuracy11

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


0.9890964446300414

In [85]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
# Precision
precision11 = precision_score(Y_test_bin_full, y_pred11)
print('Precision : %f'% precision11)

Precision : 0.856173


In [86]:
# Recall
recall11 = recall_score(Y_test_bin_full, y_pred11)
print('Recall: %f' % recall11)

Recall: 0.929430


In [87]:
# f1-score
f1_11 = f1_score(Y_test_bin_full, y_pred11)
print('F1 score: %f' % f1_11)

F1 score: 0.891299


In [88]:
roc11 = roc_auc_score(Y_test_bin_full, y_pred11)
print('ROC_AUC Score: %f' % roc11)

ROC_AUC Score: 0.960770


In [89]:
from sklearn.metrics import confusion_matrix

# confusion matrix
confusionmatrix11 = confusion_matrix(Y_test_bin_full, y_pred11)
print(confusionmatrix11)

[[388981   3093]
 [  1398  18412]]


In [90]:
tn11, fp11, fn11, tp11 = confusionmatrix11.ravel()
(tn11, fp11, fn11, tp11)

(388981, 3093, 1398, 18412)

In [93]:
#false positive rate
FPR11=fp11/(fp11+tn11)

In [98]:
FPR11*100

0.7888816907012451

In [99]:
FNR11 = fn11/(tp11+fn11)
FNR11*100

7.057041898031298

In [91]:
#now make the FN number smaller
from sklearn.utils import compute_class_weight
weights=compute_class_weight('balanced',[0,1], Y_train_bin_full)
weights

array([ 0.52546281, 10.31823989])

In [92]:
act12 = PReLU(alpha_initializer='zeros', weights=None)
model12 = Sequential()
# define the NN architecture and activation function

model12.add(Dense(496, input_dim=54, activation=Activation(act12)))
model12.add(Dropout(0.1))
model12.add(Dense(248, activation='relu'))
model12.add(Dropout(0.1))
model12.add(Dense(124, activation='relu'))
model12.add(Dropout(0.1))
model12.add(Dense(62, activation='relu'))
model12.add(Dropout(0.1))
model12.add(Dense(1))
model12.add(Activation('sigmoid'))

sgd =SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model12.compile(loss='binary_crossentropy', optimizer=sgd, metrics= ['accuracy'])
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.0001,patience=5) ## early stopping
model12.fit(X_train_bin_full, Y_train_bin_full, validation_data=(X_test_bin_full, Y_test_bin_full), epochs=100, batch_size=64, callbacks=[es],class_weight={0:1,1:3})

Epoch 1/100
25743/25743 [==============================] - 87s 3ms/step - loss: 0.0400 - accuracy: 0.9855 - val_loss: 0.0262 - val_accuracy: 0.9860
Epoch 2/100
25743/25743 [==============================] - 85s 3ms/step - loss: 0.0355 - accuracy: 0.9858 - val_loss: 0.0243 - val_accuracy: 0.9861
Epoch 3/100
25743/25743 [==============================] - 85s 3ms/step - loss: 0.0348 - accuracy: 0.9859 - val_loss: 0.0243 - val_accuracy: 0.9861
Epoch 4/100
25743/25743 [==============================] - 85s 3ms/step - loss: 0.0342 - accuracy: 0.9859 - val_loss: 0.0265 - val_accuracy: 0.9861
Epoch 5/100
25743/25743 [==============================] - 85s 3ms/step - loss: 0.0340 - accuracy: 0.9860 - val_loss: 0.0264 - val_accuracy: 0.9861
Epoch 6/100
25743/25743 [==============================] - 85s 3ms/step - loss: 0.0338 - accuracy: 0.9861 - val_loss: 0.0246 - val_accuracy: 0.9864
Epoch 7/100
25743/25743 [==============================] - 89s 3ms/step - loss: 0.0336 - accuracy: 0.9861 - val_

In [100]:
y_pred12 = model12.predict_classes(X_test_bin_full)
confusionmatrix12 = confusion_matrix(Y_test_bin_full, y_pred12)
print(confusionmatrix12)

[[386715   5359]
 [   113  19697]]


In [101]:
tn12, fp12, fn12, tp12 = confusionmatrix12.ravel()
(tn12, fp12, fn12, tp12)

(386715, 5359, 113, 19697)

In [102]:
#VERY GOOD
#false positive rate
FPR12 = fp12 / (fp12 + tn12)
FPR12 * 100

1.3668338119844723

In [103]:
FNR12 = fn12/(tp12+fn12)
FNR12*100

0.5704189803129732